# 4. Cleaning data for analysis

Dalami beberapa aspek paling bersih dari data cleaning. Pelajari tentang manipulasi string dan pencocokan pola untuk menangani data yang tidak terstruktur, dan kemudian mengeksplorasi teknik untuk menangani data yang hilang atau duplikat. Anda juga akan mempelajari keterampilan yang berharga secara terprogram memeriksa data Anda untuk konsistensi, yang akan memberi Anda keyakinan bahwa kode Anda berjalan dengan benar dan bahwa hasil analisis Anda dapat diandalkan.

## Data Types

### Converting data types

Dalam latihan ini, Anda akan melihat bagaimana memastikan semua variabel kategori dalam DataFrame dari tipe `category` mengurangi penggunaan memori.

[Dataset tips](https://github.com/mwaskom/seaborn-data/blob/master/tips.csv) telah dimuat ke dalam DataFrame yang disebut `tips`. Data ini berisi informasi tentang seberapa besar tip pelanggan, apakah pelanggan itu laki-laki atau perempuan, perokok atau tidak, dll.

Lihatlah output dari `tips.info()` di Shell IPython. Anda akan mencatat bahwa dua kolom yang harus kategorikal - `sex` dan `smoker` - bukan jenis `object`, yang merupakan cara pandas menyimpan string yang sewenang-wenang. Tugas Anda adalah mengubah dua kolom ini ke tipe `category` dan perhatikan penggunaan memori yang berkurang.

In [2]:
import pandas as pd

tips_data = 'https://assets.datacamp.com/production/repositories/666/datasets/b064fa9e0684a38ac15b0a19845367c29fde978d/tips.csv'
tips = pd.read_csv(tips_data)
tips.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
total_bill    244 non-null float64
tip           244 non-null float64
sex           244 non-null object
smoker        244 non-null object
day           244 non-null object
time          244 non-null object
size          244 non-null int64
dtypes: float64(2), int64(1), object(4)
memory usage: 13.5+ KB


In [3]:
# Convert the sex column to type 'category'
tips.sex = tips.sex.astype('category')

# Convert the smoker column to type 'category'
tips.smoker = tips.smoker.astype('category')

# Print the info of tips
print(tips.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244 entries, 0 to 243
Data columns (total 7 columns):
total_bill    244 non-null float64
tip           244 non-null float64
sex           244 non-null category
smoker        244 non-null category
day           244 non-null object
time          244 non-null object
size          244 non-null int64
dtypes: category(2), float64(2), int64(1), object(2)
memory usage: 10.3+ KB
None


**Note** : Dengan mengonversi `sex` dan `smoker` menjadi variabel kategori, penggunaan memori DataFrame turun dari 13,4 KB menjadi 10,1 KB. Ini mungkin tampak seperti perbedaan kecil di sini, tetapi ketika Anda berurusan dengan kumpulan data besar, pengurangan penggunaan memori bisa sangat signifikan!

### Working with numeric data

Jika Anda mengharapkan tipe data kolom menjadi numerik (`int` atau `float`), tetapi bukan tipe `object`, ini biasanya berarti bahwa ada nilai non numerik di kolom, yang juga menandakan data buruk.

Anda dapat menggunakan fungsi `pd.to_numeric()` untuk mengubah kolom menjadi tipe data numerik. Jika fungsi menimbulkan kesalahan, Anda dapat yakin bahwa ada bad value di dalam kolom. Anda dapat menggunakan teknik yang Anda pelajari di Bab 1 untuk melakukan beberapa analisis data eksplorasi dan menemukan bad value, atau Anda dapat memilih untuk mengabaikan atau `coerce` nilai menjadi nilai yang hilang, `NaN`.

Versi modifikasi dari dataset tips telah dimuat sebelumnya ke dalam dataFrame yang disebut `tips`. Untuk keperluan instruksional, telah diproses sebelumnya untuk memperkenalkan beberapa 'bad' data untuk Anda bersihkan. Gunakan metode `.info()` untuk menjelajahi ini. Anda akan perhatikan bahwa `total_bill` dan kolom `tip`, yang harus berupa angka, bukan tipe `object`. Tugas Anda adalah untuk memperbaikinya.

In [ ]:
# Convert 'total_bill' to a numeric dtype
tips['total_bill'] = pd.to_numeric(tips['total_bill'], errors='coerce')

# Convert 'tip' to a numeric dtype
tips['tip'] = pd.to_numeric(tips['tip'], errors='coerce')

# Print the info of tips
print(tips.info())

**Note** : Kolom `'total_bill'` dan `'tip'` dalam DataFrame ini disimpan sebagai tipe `object` karena string `'missing'` digunakan dalam kolom ini untuk menyandikan nilai yang hilang. Dengan memaksa nilai menjadi tipe numerik, mereka menjadi nilai `NaN` yang tepat.

## Using regular expressions to clean strings

### String parsing with regular expressions

*Regular expressions* atau ekspresi reguler merupakan cara ampuh untuk menentukan pola agar sesuai dengan string. Latihan ini akan membantu Anda memulai dengan menulisnya.

Saat bekerja dengan data, terkadang perlu menulis ekspresi reguler untuk mencari nilai yang dimasukkan dengan benar. Nomor telepon dalam dataset adalah bidang umum yang perlu diperiksa validitasnya. Tugas Anda dalam latihan ini adalah untuk menentukan ekspresi reguler yang cocok dengan nomor telepon AS yang sesuai dengan pola `xxx-xxx-xxxx`.

Modul ekspresi reguler dalam python adalah `re`. Saat melakukan pencocokan pola pada data, karena pola tersebut akan digunakan untuk mencocokan di beberapa baris, lebih baik untuk mengkompilasi pola terlebih dahulu menggunakan `re.compile()`, dan kemudian gunakan pola yang dikompilasi untuk mencocokkan nilai.

In [5]:
# Import the regular expression module
import re

# Compile the pattern: prog
prog = re.compile('\d{3}-\d{3}-\d{4}')

# See if the pattern matches
result = prog.match('123-456-7890')
print(bool(result))

# See if the pattern matches
result2 = prog.match('1123-456-7890')
print(bool(result2))

True
False


**Note** : Ekspresi reguler bisa terasa menantang pada awalnya, tetapi dengan latihan, Anda akan menjadi lebih baik dan lebih baik dalam menulisnya! Di sini, seperti yang diharapkan, polanya cocok dengan string pertama, tetapi bukan yang kedua.

### Extracting numerical values from strings

Mengekstraksi angka dari string adalah tugas yang umum, terutama ketika bekerja dengan data atau file log yang tidak terstruktur.

Katakanlah Anda memiliki string berikut: `'the recipe calls for 6 strawberries and 2 bananas'`.

Akan bermanfaat untuk mengekstrak `6` dan `2` dari string ini untuk disimpan untuk digunakan nanti ketika membandingkan strawberry dengan rasio pisang.

Saat menggunakan ekspresi reguler untuk mengekstrak beberapa angka (atau beberapa pencocokan pola, tepatnya), Anda dapat menggunakan fungsi `re.findall()`. Dan tidak membahas ini dalam video, tetapi mudah digunakan: Anda memberikan pola dan string untuk `re.findall()`, dan itu akan mengembalikan list yang cocok.

In [6]:
# Import the regular expression module
import re

# Find the numeric values: matches
matches = re.findall('\d+', 'the recipe calls for 10 strawberries and 1 banana')

# Print the matches
print(matches)

['10', '1']


**Note** : ekspresi reguler Anda berhasil mengekstraksi nilai numerik 10 dan 1 dari string!

### Pattern matching

Dalam latihan ini, Anda akan terus melatih keterampilan berekspresi reguler Anda. Untuk setiap string yang disediakan, tugas Anda adalah menulis pola yang sesuai untuk mencocokkannya.

String format: Tanda dolar, jumlah digit yang berubah-ubah, titik desimal, 2 digit.
* Gunakan `\$` untuk mencocokkan tanda dolar
* `\d*` untuk mencocokkan jumlah digit yang berubah-ubah
* `\.` untuk mencocokkan titik desimal
* `\d{x}` untuk mencocokkan jumlah `x` digit.
* Gunakan `[A-Z]` untuk mencocokkan huruf kapital apa pun diikuti oleh `\w*` untuk mencocokkan jumlah karakter alfanumerik yang berubah-ubah.

In [7]:
# Write the first pattern
pattern1 = bool(re.match(pattern='\d{3}-\d{3}-\d{4}', string='123-456-7890'))
print(pattern1)

# Write the second pattern
pattern2 = bool(re.match(pattern='\$\d*\.\d{2}', string='$123.45'))
print(pattern2)

# Write the third pattern
pattern3 = bool(re.match(pattern='[A-Z]\w*', string='Australia'))
print(pattern3)

True
True
True


## Using functions to clean data

### Custom functions to clean data

Sekarang Anda akan berlatih menulis fungsi untuk membersihkan data.

Kumpulan data tips telah dimuat sebelumnya ke dalam DataFrame yang disebut `tips`. Ini memiliki kolom `'sex'` yang berisi nilai `'Male'` atau `'Female'`. Tugas Anda adalah menulis fungsi yang akan mengkode ulang `'Female'` ke `0`, `'Male'` ke `1`, dan mengembalikan `np.nan` untuk semua entri `'sex'` yang bukan `'Female'` atau `'Male'`.

Pengodean ulang variabel seperti ini adalah tugas pembersihan data yang umum. Fungsi menyediakan mekanisme bagi Anda untuk mengabstraksi bit kode yang kompleks serta menggunakan kembali kode. Ini membuat kode Anda lebih mudah dibaca dan lebih sedikit kesalahan.

Anda dapat menggunakan metode `.apply()` untuk menerapkan fungsi di seluruh baris atau kolom DataFrames. Namun, perhatikan bahwa setiap kolom DataFrame adalah Pandas Series. Fungsi juga dapat diterapkan di seluruh Series. Di sini, Anda akan menerapkan fungsi Anda di kolom `'sex'`.

In [10]:
# Define recode_gender()
def recode_gender(gender):

    # Return 0 if gender is 'Female'
    if gender == 'Female':
        return 0
    
    # Return 1 if gender is 'Male'    
    elif gender == 'Male':
        return 1
    
    # Return np.nan    
    else:
        return np.nan

# Apply the function to the sex column
tips['recode'] = tips.sex.apply(recode_gender)

# Print the first five rows of tips
tips.head()

,total_bill,tip,sex,smoker,day,time,size,recode
0,16.99,1.01,Female,No,Sun,Dinner,2,0
1,10.34,1.66,Male,No,Sun,Dinner,3,1
2,21.01,3.50,Male,No,Sun,Dinner,3,1
3,23.68,3.31,Male,No,Sun,Dinner,2,1
4,24.59,3.61,Female,No,Sun,Dinner,4,0


**Note** : Untuk pengodean ulang sederhana, Anda juga dapat menggunakan metode `replace`. Anda juga dapat mengubah kolom menjadi tipe kategorikal.

### Lambda functions

Sekarang Anda akan diperkenalkan dengan fitur Python yang kuat yang akan membantu Anda membersihkan data Anda lebih efektif: fungsi lambda. Alih-alih menggunakan sintaks `def` yang Anda gunakan dalam latihan sebelumnya, fungsi lambda membiarkan Anda membuat fungsi sederhana, satu-baris.

Misalnya, inilah fungsi yang menguadratkan variabel yang digunakan dalam metode `.apply()`:

<pre>
def my_square(x):
    return x ** 2

df.apply(my_square)
</pre>

Kode yang setara menggunakan fungsi lambda adalah:

<pre>df.apply(lambda x: x ** 2)</pre>

Fungsi lambda mengambil satu parameter - variabel `x`. Fungsi itu sendiri hanya kuadrat `x` dan mengembalikan hasilnya, yang merupakan apa pun yang dievaluasi oleh satu baris kode. Dengan cara ini, fungsi lambda dapat membuat kode Anda ringkas dan Pythonic.

Kumpulan data tips telah dimuat sebelumnya ke dalam DataFrame yang disebut `tips`. Tugas Anda adalah membersihkan kolom `'total_dollar'` dengan menghapus tanda dolar. Anda akan melakukan ini menggunakan dua metode berbeda: Dengan metode `.replace()`, dan dengan *regular expressions*. 

In [ ]:
# Write the lambda function using replace
tips['total_dollar_replace'] = tips.total_dollar.apply(lambda x: x.replace('$', ''))

# Write the lambda function using regular expressions
tips['total_dollar_re'] = tips.total_dollar.apply(lambda x: re.findall('\d+\.\d+', x)[0])

# Print the head of tips
print(tips.head())

## Duplicate and missing data

### Dropping duplicate data

Data rangkap menyebabkan berbagai masalah. Dari sudut pandang kinerja, mereka menggunakan jumlah memori yang tidak perlu dan menyebabkan perhitungan yang tidak perlu dilakukan saat memproses data. Selain itu, mereka juga dapat membiaskan hasil analisis.

Dataset yang berisi kinerja lagu di tangga lagu Billboard telah dimuat sebelumnya ke dalam DataFrame yang disebut `billboard`. Lihat kolomnya di Shell IPython. Tugas Anda dalam latihan ini adalah untuk mengatur ulang DataFrame ini dan kemudian menghapus semua baris duplikat.

In [ ]:
# Create the new DataFrame: tracks
tracks = billboard[['year', 'artist', 'track', 'time']]

# Print info of tracks
print(tracks.info())

# Drop the duplicates: tracks_no_duplicates
tracks_no_duplicates = tracks.drop_duplicates()

# Print info of tracks
print(tracks_no_duplicates.info())

### Filling missing data

Di sini, Anda akan kembali ke dataset `airquality` dari Bab 2. Telah dimuat sebelumnya ke dalam DataFrame `airquality`, dan memiliki nilai yang hilang untuk Anda praktikkan mengisi. Jelajahi `airquality` di IPython Shell untuk memeriksa kolom mana yang memiliki nilai yang hilang.

Jarang memiliki dataset (dunia nyata) tanpa nilai yang hilang, dan penting untuk mengatasinya karena perhitungan tertentu tidak dapat menangani nilai yang hilang sementara beberapa perhitungan secara default akan melewatkan nilai-nilai yang hilang.

Juga, memahami berapa banyak data yang hilang yang Anda miliki, dan memikirkan dari mana asalnya sangat penting untuk membuat interpretasi data yang tidak bias.

In [11]:
import pandas as pd

data = 'https://assets.datacamp.com/production/repositories/666/datasets/c16448e3f4219f900f540c455fdf87b0f3da70e0/airquality.csv'
airquality = pd.read_csv(data)
airquality.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153 entries, 0 to 152
Data columns (total 6 columns):
Ozone      116 non-null float64
Solar.R    146 non-null float64
Wind       153 non-null float64
Temp       153 non-null int64
Month      153 non-null int64
Day        153 non-null int64
dtypes: float64(3), int64(3)
memory usage: 7.3 KB


In [12]:
# Calculate the mean of the Ozone column: oz_mean
oz_mean = airquality.Ozone.mean()

# Replace all the missing values in the Ozone column with the mean
airquality['Ozone'] = airquality.Ozone.fillna(oz_mean)

# Print the info of airquality
print(airquality.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153 entries, 0 to 152
Data columns (total 6 columns):
Ozone      153 non-null float64
Solar.R    146 non-null float64
Wind       153 non-null float64
Temp       153 non-null int64
Month      153 non-null int64
Day        153 non-null int64
dtypes: float64(3), int64(3)
memory usage: 7.3 KB
None


**Note** : Tidak ada lagi nilai yang hilang di kolom Ozon dari DataFrame ini!

## Testing with asserts

### Testing your data with asserts

Di sini, Anda akan berlatih menulis pernyataan `assert` menggunakan dataset Ebola dari bab sebelumnya untuk memeriksa secara terprogram nilai yang hilang dan untuk mengonfirmasi bahwa semua nilai positif. Dataset telah dimuat sebelumnya ke dalam DataFrame yang disebut `ebola`.

Dalam video, Anda melihat Dan menggunakan metode `.all()` bersama dengan metode DataFrame `.notnull()` untuk memeriksa nilai yang hilang dalam kolom. Metode `.all()` mengembalikan `True` jika semua nilai bernilai `True`. Saat digunakan pada DataFrame, ia mengembalikan serangkaian Boolean - satu untuk setiap kolom di DataFrame. Jadi, jika Anda menggunakannya pada DataFrame, seperti dalam latihan ini, Anda perlu mem-chain metode `.all()` lainnya sehingga Anda hanya mengembalikan satu nilai `True` atau `False`. Saat menggunakan ini dalam pernyataan assert, tidak ada yang akan dikembalikan jika pernyataan assert benar: Ini adalah bagaimana Anda dapat mengkonfirmasi bahwa data yang Anda periksa valid.

Catatan: Anda dapat menggunakan `pd.notnull(df)` sebagai alternatif dari `df.notnull()`.

In [14]:
data = 'https://assets.datacamp.com/production/repositories/666/datasets/6da83b3d2017245217d35989960184234a6c4e7f/ebola.csv'
ebola = pd.read_csv(data)
ebola.head()

,Date,Day,Cases_Guinea,Cases_Liberia,Cases_SierraLeone,Cases_Nigeria,Cases_Senegal,Cases_UnitedStates,Cases_Spain,Cases_Mali,Deaths_Guinea,Deaths_Liberia,Deaths_SierraLeone,Deaths_Nigeria,Deaths_Senegal,Deaths_UnitedStates,Deaths_Spain,Deaths_Mali
0,1/5/2015,289,2776.0,NaN,10030.0,NaN,NaN,NaN,NaN,NaN,1786.0,NaN,2977.0,NaN,NaN,NaN,NaN,NaN
1,1/4/2015,288,2775.0,NaN,9780.0,NaN,NaN,NaN,NaN,NaN,1781.0,NaN,2943.0,NaN,NaN,NaN,NaN,NaN
2,1/3/2015,287,2769.0,8166.0,9722.0,NaN,NaN,NaN,NaN,NaN,1767.0,3496.0,2915.0,NaN,NaN,NaN,NaN,NaN
3,1/2/2015,286,NaN,8157.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3496.0,NaN,NaN,NaN,NaN,NaN,NaN
4,12/31/2014,284,2730.0,8115.0,9633.0,NaN,NaN,NaN,NaN,NaN,1739.0,3471.0,2827.0,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Assert that there are no missing values
assert pd.notnull(ebola).all().all()

# Assert that all values are >= 0
assert (ebola >= 0).all().all()

**Note** : Karena pernyataan assert tidak menimbulkan kesalahan, Anda dapat yakin bahwa tidak ada nilai yang hilang dalam data dan bahwa semua nilai `>=` 0!

*Data versi DataCamp telah dibersihkan, sehingga assert tidak menimbulkan kesalahan*